In [69]:
import string
from pathlib import Path
from collections import defaultdict
import json
import numpy as np
import math
from tqdm import tqdm

file = open('./input.txt', 'r')

map = []

directions = ['N','S','W','E']


for line in file.readlines():
    line = line.replace('\n','')
    
    map_line = []
    for el in line:
        if el == '.':
            map_line.append(0)
        else:
            assert el=='#', el
            map_line.append(1)
    map.append(map_line)

map_widths = [len(row) for row in map]
max_width = max(map_widths)
for i,row in enumerate(map):
    while len(row)<max_width:
        row.append(0)
    map[i] = row

print(f'map height: {len(map)}')
print(f'map min and max width: {min(map_widths)}, {max(map_widths)}')
# print(f'moves length: {moves_len}')
# print(f'facing directions: {facing_dir}')

map height: 75
map min and max width: 75, 75


In [70]:
margin=100
map_array = np.zeros((len(map)+2*margin,min(map_widths)+2*margin))
map_array[margin:-margin,margin:-margin] = np.array(map)
map = map_array.copy()

print(f'map shape = {map.shape}')


map shape = (275, 275)


In [71]:
def new_elf_pose(y,x,map,directions):

    assert map[y,x] == 1
    if np.sum(map[y-1:y+2,x-1:x+2])==1:
        # print(f'elf in ({x},{y}) no need to move')
        return y,x,False
    for d in directions:
        if d == 'N' and np.sum(map[y-1,x-1:x+2])==0:
            # print(f'elf in ({x},{y}) can move north')
            return y-1,x,True
        if d == 'S' and np.sum(map[y+1,x-1:x+2])==0:
            # print(f'elf in ({x},{y}) can move south')
            return y+1,x,True
        if d == 'E' and np.sum(map[y-1:y+2,x+1])==0:
            # print(f'elf in ({x},{y}) can move east')
            return y,x+1,True
        if d == 'W' and np.sum(map[y-1:y+2,x-1])==0:
            # print(f'elf in ({x},{y}) can move west')
            return y,x-1,True
    # print(f'elf in ({x},{y}) cannot move in any dir')
    return y,x,True

def propose_pos(map, directions):

    pos_y, pos_x = np.where(map==1)
    proposed_pos = []
    one_moves = False
    num_moves = 0
    for y,x in zip(pos_y, pos_x):

        new_y, new_x, move = new_elf_pose(y,x,map,directions)
        # print(f'elf in ({x},{y}) should move to ({new_x},{new_y})')
        if move:
            one_moves=True
            num_moves += 1

        proposed_pos.append((y,x,new_y,new_x))

    return proposed_pos, one_moves, num_moves

def update_pos(proposed_pos, map):

    num_elves = np.sum(map)
    new_pos = [(p[2],p[3]) for p in proposed_pos]
    final_pos = []
    
    for p in proposed_pos:

        if new_pos.count((p[2],p[3]))>=2:
            # print(f'elf in ({p[0]},{p[1]}) stay there')
            final_pos.append((p[0],p[1]))
        else:
            # print(f'elf in ({p[0]},{p[1]}) moves to ({p[2]},{p[3]})')
            final_pos.append((p[2],p[3]))

    map = np.zeros_like(map)
    for p in final_pos:
        map[p[0],p[1]] = 1

    assert np.sum(map)==num_elves
    return map

        

In [72]:
def draw_map(map):

    repr=''
    for i in range(map.shape[0]):
        for j in range(map.shape[1]):
            if map[i,j]==1:
                repr += '#'
            else:
                repr += '.'
        repr += '\n'

    print(repr)

In [73]:
# for idx in range(10):
    
#     proposed_pos, one_moves = propose_pos(map, directions)
#     map = update_pos(proposed_pos, map)

#     direct = directions.pop(0)
#     directions.append(direct)

#     min_y = np.min(np.where(map==1)[0])    
#     max_y = np.max(np.where(map==1)[0])    
#     min_x = np.min(np.where(map==1)[1])    
#     max_x = np.max(np.where(map==1)[1])

#     spaces = np.sum(np.where(map[min_y:max_y+1,min_x:max_x+1]==0,1,0))

#     print(f'in rectangle [{min_y}:{max_y},{min_x}:{max_x}] there are {spaces} spaces')

# draw_map(map[min_y-1:max_y+2,min_x-1:max_x+2])

In [74]:
one_moves = True
round = 0

print(f'map shape = {map.shape}')
while one_moves:
    round +=1
    
    proposed_pos, one_moves, num_moves = propose_pos(map, directions)
    map = update_pos(proposed_pos, map)

    direct = directions.pop(0)
    directions.append(direct)

    min_y = np.min(np.where(map==1)[0])    
    max_y = np.max(np.where(map==1)[0])    
    min_x = np.min(np.where(map==1)[1])    
    max_x = np.max(np.where(map==1)[1])

    spaces = np.sum(np.where(map[min_y:max_y+1,min_x:max_x+1]==0,1,0))

    print(f'ROUND {round}: {num_moves} moves, in rectangle [{min_y}:{max_y},{min_x}:{max_x}] there are {spaces} spaces')

draw_map(map[min_y-1:max_y+2,min_x-1:max_x+2])

map shape = (275, 275)
ROUND 1: 2789 moves, in rectangle [99:175,99:175] there are 3122 spaces
ROUND 2: 2770 moves, in rectangle [99:176,98:176] there are 3355 spaces
ROUND 3: 2762 moves, in rectangle [98:176,97:176] there are 3513 spaces
ROUND 4: 2743 moves, in rectangle [97:177,97:177] there are 3754 spaces
ROUND 5: 2741 moves, in rectangle [97:177,97:177] there are 3754 spaces
ROUND 6: 2736 moves, in rectangle [97:178,96:177] there are 3917 spaces
ROUND 7: 2727 moves, in rectangle [97:178,95:178] there are 4081 spaces
ROUND 8: 2704 moves, in rectangle [96:178,96:178] there are 4082 spaces
ROUND 9: 2693 moves, in rectangle [96:178,96:178] there are 4082 spaces
ROUND 10: 2694 moves, in rectangle [96:179,95:178] there are 4249 spaces
ROUND 11: 2698 moves, in rectangle [96:179,95:179] there are 4333 spaces
ROUND 12: 2684 moves, in rectangle [96:179,95:179] there are 4333 spaces
ROUND 13: 2687 moves, in rectangle [95:179,95:179] there are 4418 spaces
ROUND 14: 2671 moves, in rectangle [9